## Lab Assignment 2
### Data Mining 7331 Section 403
---
- Brian Coari
- Stephen Merritt
- Cory Thigpen
- Quentin Thomas

This dataset tracks all Olympians' medal results from 1896-2016, as well as various physical attributes such as `Sex`, `Age`, `Weight`, and `Height`. In this lab we will (1) predict podium finishers using binary classification and (2) predict the sport a podium-finishing athlete played using multinomial classification.

### Data Preparation

#### Section 1

- (10pts) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

---

In [11]:
import numpy as np
import pandas as pd
df = pd.read_csv('~/olympics/data/athletes_cleaned_merged.csv')
df.head()

,Unnamed: 0,NOC,Country,ID,Name,Sex,Age,Height,Weight,Team,Games,Year,Season,City,Sport,Event,Medal
0,872,AFG,Afghanistan,502.0,Ahmad Shah Abouwi,M,25,176,73,Afghanistan,1956 Summer,1956,Summer,Melbourne,Hockey,Hockey,No Medal
1,1950,AFG,Afghanistan,1076.0,Jammal-ud-Din Affendi,M,28,176,73,Afghanistan,1936 Summer,1936,Summer,Berlin,Hockey,Hockey,No Medal
2,1992,AFG,Afghanistan,1101.0,Mohammad Anwar Afzal,M,23,177,73,Afghanistan,1948 Summer,1948,Summer,London,Football,Football,No Medal
3,3129,AFG,Afghanistan,1745.0,Mohammad Aktar,M,17,156,48,Afghanistan,1980 Summer,1980,Summer,Moskva,Wrestling,"Light-Flyweight, Freestyle",No Medal
4,8410,AFG,Afghanistan,4628.0,Mohammad Daoud Anwary,M,22,172,76,Afghanistan,1964 Summer,1964,Summer,Tokyo,Wrestling,"Bantamweight, Freestyle",No Medal


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266954 entries, 0 to 266953
Data columns (total 17 columns):
Unnamed: 0    266954 non-null int64
NOC           266954 non-null object
Country       266954 non-null object
ID            266954 non-null float64
Name          266954 non-null object
Sex           266954 non-null object
Age           266954 non-null int64
Height        266954 non-null int64
Weight        266954 non-null int64
Team          266954 non-null object
Games         266954 non-null object
Year          266954 non-null int64
Season        266954 non-null object
City          266954 non-null object
Sport         266954 non-null object
Event         266954 non-null object
Medal         266954 non-null object
dtypes: float64(1), int64(5), object(11)
memory usage: 34.6+ MB


The intial data set contained the following data of different types:

| Nominal | Ordinal | Numeric | Binary |
| ------- | ------- | ------- | ------ |
| Games   | City    | Age     |        |
| Name    | Event   | Height  |        |
| NOC     | ID      | Weight  |        |
| Season  | Medal   | Year    |        |
| Sex     |         |         |        |
| Team    |         |         |        |

To prepare the lab, we brought population data into the data set and changed all National Olympic Committee or `NOC` data points to match their corresponding `Country`. For countries without entries in the worldbank URL we hard-coded the populations. There were 106 observations in the data set not associated with a country, so we dropped them. These were `Refugee Athletes` and `Individual Olympic Athletes`. `Sex`, `Season`, and `Medal` were converted to numeric factors and `ID`, `NOC`, `Name`, `Team`, and `Games` were dropped due to lack of relevancy.

In [13]:
df_pop = pd.read_csv('~/olympics/data/POP_TOTAL.csv', encoding = "ISO-8859-1")
df_pop = df_pop[['Country Code', '2015']]
#Change all Russia NOCs
df['NOC'] = np.where(df['NOC'] == 'EUN', 'RUS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'URS', 'RUS', df['NOC'])
#Change all Australia NOCs
df['NOC'] = np.where(df['NOC'] == 'ANZ', 'AUS', df['NOC'])
#Change all German NOCs
df['NOC'] = np.where(df['NOC'] == 'FRG', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'GDR', 'GER', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SAA', 'GER', df['NOC'])
#Change all Congo NOCs
df['NOC'] = np.where(df['NOC'] == 'CGO', 'COD', df['NOC'])
#Change all Czech NOCs
df['NOC'] = np.where(df['NOC'] == 'BOH', 'CZE', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'TCH', 'CZE', df['NOC'])
#Change all Yemen NOCs
df['NOC'] = np.where(df['NOC'] == 'YAR', 'YEM', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'YMD', 'YEM', df['NOC'])
#Change all Greek NOCs
df['NOC'] = np.where(df['NOC'] == 'CRT', 'GRE', df['NOC'])
#Change all Zimbabwe NOCs
df['NOC'] = np.where(df['NOC'] == 'RHO', 'ZIM', df['NOC'])
#Change all Malaysia NOCs
df['NOC'] = np.where(df['NOC'] == 'MAL', 'MAS', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'NBO', 'MAS', df['NOC'])
#Change all Vietnam NOCs
df['NOC'] = np.where(df['NOC'] == 'VNM', 'VIE', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'WIF', 'TTO', df['NOC'])
#Change all Trinidad and Tobego NOCs
df['NOC'] = np.where(df['NOC'] == 'UAR', 'SYR', df['NOC'])
#Change all Serbian NOCs
df['NOC'] = np.where(df['NOC'] == 'SCG', 'YUG', df['NOC'])
df['NOC'] = np.where(df['NOC'] == 'SRB', 'YUG', df['NOC'])
#Change all Canada NOCs
df['NOC'] = np.where(df['NOC'] == 'NFL', 'CAN', df['NOC'])
df_pop.rename(index = str, columns = {'Country Code' : 'NOC', '2015': '2015 Population'}, inplace = True)
df_pop_merge = pd.merge(left = df, right = df_pop, how = 'left', on=['NOC'], left_index=False)
df_pop_merge.loc[df_pop_merge['Country'] == 'Taiwan', '2015 Population'] =  23485755
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Vincent', '2015 Population'] =  109643
df_pop_merge.loc[df_pop_merge['Country'] == 'Palestine', '2015 Population'] =  4817000
df_pop_merge.loc[df_pop_merge['Country'] == 'Cook Islands', '2015 Population'] =  17459
df_pop_merge.loc[df_pop_merge['Country'] == 'Eritrea', '2015 Population'] =  4846976
df_pop_merge.loc[df_pop_merge['Country'] == 'Saint Kitts', '2015 Population'] =  54821
df_pop_mergena = df_pop_merge[df_pop_merge['2015 Population'].isnull()]
#Drop the 106 observations not associated with a country 
df_pop_merge = df_pop_merge.dropna(how = 'any')

In [15]:
#Calculate the population proportion for each country
df_pop = df_pop_merge.groupby(['Country'])['2015 Population'].mean().reset_index()
df_pop['Population_Prop'] = df_pop['2015 Population'].transform(lambda x: x / (x.sum()))

#Calculate BMI for each athlete
def calculate_bmi(df):
    return (df[1]/(df[0] * df[0])) * 10000
df_pop_merge['BMI'] = df_pop_merge[['Height', 'Weight']].apply(calculate_bmi, axis=1)

#Merge dataframes with Population Propotion information
df_pop_merge = pd.merge(left = df_pop_merge, right = df_pop, how = 'left', on=['Country'], 
                        left_index=False)
#Create Binary Medal, Season, and Sex Classes
df_pop_merge['Medal'] = np.where(df_pop_merge.Medal == 'No Medal', 0, 1)
df_pop_merge['Season'] = np.where(df_pop_merge.Season == 'Summer', 1, 0)
df_pop_merge['Sex'] = np.where(df_pop_merge.Sex == 'M', 1, 0)

In [16]:
%%time

prev_medals = df_pop_merge.groupby(['Name', 'Year'])['Medal'].sum()
prev_medals = prev_medals.reset_index()
prev_medals.columns = ['Name', 'Year', 'Medal_Count']
prev_medals['Previous_Medals'] = prev_medals['Medal_Count']

for i in range(2, len(prev_medals)):
    if prev_medals.iloc[i]['Name'] == prev_medals.iloc[i-1]['Name']:
        if prev_medals.iloc[i-1]['Medal_Count'] != 0:
            prev_medals['Previous_Medals'].iloc[i] = 1
        else:
            prev_medals['Previous_Medals'].iloc[i] = 0
    else:
        prev_medals['Previous_Medals'].iloc[i] = 0

/Users/Steve/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


CPU times: user 1h 38min 49s, sys: 12.6 s, total: 1h 39min 2s
Wall time: 1h 39min 11s


In [17]:
#Merge prev_medals into existing pop_merge dataset to add Previous_Medals feature
df_prev_medal_merge = pd.merge(left = df_pop_merge, right = prev_medals, how = 'left', on = ['Name', 'Year'])
df_pop_merge = df_prev_medal_merge[['Country', 'Sex', 'Age', 'Height', 'Weight', 'Year', 'Season', 'Sport', 'Event', 'BMI', 
                             'Population_Prop', 'Medal', 'Previous_Medals']]

#Write dataframe to .csv file in order to preserve the data and avoid Feature Engineering code in future iterations 
df_pop_merge.to_csv('~/olympics/data/pop_merge.csv')

In [18]:
#Create X and y vectors for sklearn pre-processing steps
df = pd.read_csv('~/olympics/data/pop_merge.csv')

y = df['Medal']
X = df.drop(['Medal', 'Unnamed: 0'], axis = 1)

print(X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266848 entries, 0 to 266847
Data columns (total 12 columns):
Country            266848 non-null object
Sex                266848 non-null int64
Age                266848 non-null int64
Height             266848 non-null int64
Weight             266848 non-null int64
Year               266848 non-null int64
Season             266848 non-null int64
Sport              266848 non-null object
Event              266848 non-null object
BMI                266848 non-null float64
Population_Prop    266848 non-null float64
Previous_Medals    266848 non-null int64
dtypes: float64(2), int64(7), object(3)
memory usage: 24.4+ MB
None


`Country`, `Sport`, and `Event` had too many variables for binary coding, so one-hot encoding was performed.

In [19]:
X = pd.get_dummies(X, columns = ['Country', 'Event', 'Sport'])
print(X.describe())
print(y.describe())

                 Sex            Age         Height         Weight  \
count  266848.000000  266848.000000  266848.000000  266848.000000   
mean        0.722786      25.330338     175.420948      71.017542   
std         0.447624       5.801661       9.869066      13.330644   
min         0.000000      10.000000     127.000000      25.000000   
25%         0.000000      21.000000     169.000000      62.000000   
50%         1.000000      24.000000     176.000000      71.000000   
75%         1.000000      28.000000     182.000000      78.000000   
max         1.000000      72.000000     226.000000     214.000000   

                Year         Season            BMI  Population_Prop  \
count  266848.000000  266848.000000  266848.000000    266848.000000   
mean     1979.016369       0.818185      22.904691         0.013419   
std        29.560519       0.385693       2.718862         0.028553   
min      1896.000000       0.000000       8.360954         0.000001   
25%      1960.000000   

In [20]:
from sklearn.preprocessing import StandardScaler
#Standardize X
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

#### Section 2

- (5pts) Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

---

Three variables were added to the data set: `Population_Prop`, `BMI`, and `Previous_Medals`. The `Population_Prop` variable was added to represent the percentage of the overall world population for a given `Country`. Athlete `BMI` was added to the dataframe where $\omega$ represents `Weight` in kilograms, and $\eta$ represents `Height` in centimeters, and $C$ is a constant at `10,000`:

$$
BMI = \left(\frac{\omega}{\eta \eta}\right) C
$$

`Previous_Medals` was created as a binary representation if an athlete had won a medal at a previous Olympic games (indicated by a `1`). The `Medal` variable was converted from ordinal to binary, where `Gold`, `Silver`, or `Bronze` were given a `1` and `No Medal` performers were given a `0`. `Season` and `Sex` were also converted to binary. 

One-hot encoding was performed for binarization of the remaining nominal variables(`Country`, `Event`, and `Sport`) so they could be included as features to train the classification model.

The final data set contains the following data of different types:

| Nominal | Ordinal | Numeric         | Binary          |
| ------- | ------- | --------------- | --------------- | 
|         |         | Age             | Country         |
|         |         | BMI             | Event           |
|         |         | Height          | Medal           |
|         |         | Population_Prop | Previous_Medals |
|         |         | Weight          | Season          |
|         |         | Year            | Sex             |
|         |         |                 | Sport           |

The binary classification model will predict podium finishers and using multinomial classification we will predict one of 54 possible sports a podium-finishing athlete played.

### Modeling and Evaluation

#### Section 1

- (10 pts) Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

---

#### Section 2

- (10pts) Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

---

In [ ]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=0)

#### Section 3

- (20pts) Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

---

In [ ]:
# Classifer Estimator created and edited from Professor Drew's NC school repository
def EvaluateClassifierEstimator2(classifierEstimator, X, y, cv):
    
    #Perform cross validation 
    from sklearn.model_selection import cross_val_predict
    predictions = cross_val_predict(classifierEstimator, X, y, cv=cv)
    
    #model evaluation 
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
    
    #pass true test set values and predictions to classification_report
    classReport = classification_report(y, predictions)
    confMat = confusion_matrix(y, predictions)
    acc = accuracy_score(y, predictions)

    
    print(classReport)
    print(confMat)
    print(acc)

In [ ]:
#Logisitic regression 10-fold cross-validation 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression()


parameters = { 'penalty':['l2','l1']
              ,'C': [0.001, 0.01, 0.1, 1, 10, 100]
              ,'class_weight': ['balanced','none']
              ,'random_state': [0]
              ,'solver': ['sag', 'saga']
              ,'max_iter':[100,500]
             }

#Create a grid search object using the  
regGS = GridSearchCV(estimator=lr
                   , n_jobs=-1 #Use all possible cores to run jobs in parallel
                   , verbose=True
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring='f1')

#Perform hyperparameter search to find the best combination of parameters for our data
regGS.fit(X_std, Y)

In [ ]:
lr_clf = GS.best_estimator_

In [ ]:
EvaluateClassifierEstimator2(lr_clf, X_std, y, 10)

In [ ]:
lr_clf = lr_clf.fit(X_std, y)

In [ ]:
from sklearn.externals import joblib
gs_lr_model = 'gs_lr_model.joblib.pkl'
_ = joblib.dump(lr_clf, gs_lr_model, compress = 9)
#This will save the model in your working directory

In [ ]:
clf2 = joblib.load(gs_lr_model)
clf2

#### Section 4

- (10pts) Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

---

#### Section 5

- (10pts) Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.

---

#### Section 6

- (10pts) Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

---

### Deployment

- (5pts) How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

---

These predictions could be used for predicting medal winners in future Olympics based on variables such as `BMI`, `Height`, `Weight`, or `Sex`; however, this is a bit impractical due to the lack of discernable difference in skillset. Individuals with similar characteristics do not necessarily perform the same. To make podium predictions more useful we would need a skill level metric, but skill levels change over time. Consequently, this data set is better served for informational or educational purposes.

To teach an individual or organization about <i>data science</i>, <i>machine learning</i>, or <i>data mining</i> we must first produce something relatable and <u>this</u> data set is something people understand, so as we explain what classifications we are making the content becomes understandable. From there we can dive into data that is more relavent to that individual or organization.

Binary and multinomial classification have numerous use cases across industries. For example:

| Industry      | Use Case                |
| ------------- | ----------------------- |
| Advertising   | Targeted advertising    |
| Energy        | Prediction of dry well  |
| Manufacturing | Risk of machine failure |
| Sports        | Run or pass in football |
| Telecom       | Customer retention      |
| Underwriting  | Risk level of borrower  |

To <b>measure the value</b> of any prediction model we would first need to determine if the model classifies better than best guess. If so we would need to develop a business case to understand the cost implications. To do this we would quantify risk reduction, revenue increases, changes in working capital, and/or cost decreases that were a direct result of the model. Next we would determine the cost of implementing the model and perform a discounted cash flow analysis to ensure there was a positive net present value.

This type of model could be useful for predicting what sport someone may be best suited to play. Attributes could be inputs that lead to a recommended sport for a person of a particular `Age`, `Height`, `Weight`, `BMI`, `Sex`, and `Country`. More broadly, to <b>deploy</b> a model under any of the aforementithis type of model could be useful for predicting what sport someone may be best suited to play. oned use cases we would need to understand the architecture environment and determine how to use APIs to connect to the classification model. Once this is established the predictor could be integrated with an existing system or set up on a clean build.

It may take many hours to <b>train</b> a prediction model, so it would be impractical to retrain a model in real-time using a relational database. Instead, the frequency for updating the prediction model would need to be determined by the nature of the use case. Using the telecom example from above, customer retention prediction models would likely run monthly to align with billing and usage limits. For the Olympics, this model could be updated annually/seasonally with each World Championship or every two years after the Olympic Games.

### Exceptional Work

- (10pts) You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

---

### Automated Search for Optimal Parameters using Evolutionary Algorithms.

Grid Search has been used in order to search for parameters in most machine learning problems. The problem with Gridsearch is it only tries a small subset of parameters that can be used to fit models. Grid search does not find the most optimal, or near optimal solution because it does not search enough parameters to know whether or not it has reached an optimal solution. For this reason we decided to develop a framework using an Evolutionary approach to finding optimal or near optimal for hyper parameters regardless of the model.

`Benefits of Evolutionary Algorithms`: Evolutionary algorithms try a LARGE amount of parameter values which can dwarf the amount of parameters that Gridsearch tries. Each time the algorithm repeats it selects only the best solutions that best maximizes, or minimizes the objective function and passes that information to the next generation to find the best parameters in the new generation. This generational cycle of locating the best parameters continue until the most optimal, or near optimal solution has been found. In short...

1. Tries more values than grid search
2. Only considers the best values, and from those values create even better values.
3. Can be used for ANY ML algorithm.

#### How the algorithm works

The rules for the algorithm is straight forward. In our case we want to find the best parameters for a model that help us MINIMIZE our objective function. Our objective function is our model's error rate. We would like to find parameters for models that best predict podium finishers with the smallest amount of error.

![Genetic Algorithm Image](images/genetic_algorithm.png "Title")

Taking this approach to parameter search ensures that we are left with only the best of the best solutions and the even better we don't have to manual perform the searches ourselves.

#### The Goal

In order to start the algorithm we first must determine what we would like to search for. In this example, we will use the `Bernoulli Naive Bayes` model as it was found to be our best performing model at 83% accuracy. Our goal however, is to determine if we can do better than our grid search.

The `Bernoulli Naive Bayes` has 2 hyperparameters that a user must select in order to perform a fit on the model. The issue here is that we have no clue which ones to try, and how they will affect our results. This is where the genetic algorithm can help us. We will use the genetic algorithm in order to find $\alpha$ and $\beta$ which will represent our two hyper parameters `ALPHA` and `BINARIZE` values. We would like to find the best of these two values that will give us the lowest error rate for this model.

#### Representing Alpha and Binarize as binaries

In order to begin our search we will start with vector of 30 slots. This vector will have 15 slots that represent the $\alpha$ chromosome and 15 slots that represent the $\beta$ or Binarize chromosome.

![Parameter Representation](images/vector_rep.png "Title")

The inital states for the parameters will be represented as a random binary to the algorithm, and the algorithms job will be to find the best binary representation that best fits our objective function which is minimizing model error.

#### Implementation

Next, we will see a detailed implementation of our framework.

In [1]:
import pandas as pd
import numpy as np
import random as rd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.naive_bayes import BernoulliNB
from multiprocessing import Pool
from tqdm import tqdm

/Users/osknowsys/anaconda3/envs/olympics/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
data = pd.read_csv("data/pop_merge.csv",index_col=False)
data = data.sample(frac=1)

y = data['Medal']
X = data.drop(['Medal', 'Unnamed: 0'], axis = 1)

X = pd.get_dummies(X, columns = ['Country', 'Event', 'Sport'])

sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

Above, we find our olympic dataset and prepare our variables. We normalize all our numeric values, and create dummy variables for our qualitative variables.

In [3]:
starting_gene = np.array([1,1,1,1,0,0,1,0,1,0,0,1,0,1,1,1,1,1,0,0,1,1,1,1,0,0,1,0,1,0])

Above our starting gene will be our random binary representation of both hyper parameters as one vector. Each hyper parameter will be represented by 15 bits each.

In order for our algorithm to work we will need several values created in order to keep track of the progress. Below is a description of values our framework uses...

`Probability Of Crossover`: This is the probability that a solution should be crossover with another solution. We want this to always happen so it is 1.

`Probability of Mutation`: This is the probability that there will be a change to a given solution. We want this value to be do to random chance, so in order to mutate the random chance must be greater than a value of `0.2`.

`Population`: This represents the number of random solutions that will be generated for each generation. Our algorithm selects solutions from this group.

`Generation`: This is the number of generations that our algorithm needs to try to find a good solution. This can be as high as we want, but compute resources need to be really good in order to handle a large number of generations. in our case we will only look for solutions for `10` generations.

`kfold`: The number of folds to use in the training process. We will set this to 3.

`range for alpha`: This is the search range we would like our algorithm to search in. This can be as wide as we want it as there are many solutions that will be explored. In our case we use values that make sense for alpha from `0.0` to `1.0`.

`range for binarize`: This is the search range we would like our algorithm to search in for binarize. `0.0` to `1.01`.

`X`: The dataset

`y`: the target in this case podium finish or no podium finish

`L`: The total length of the chromosome binary vector

`Winners`: The list of the best results found throughout all the generations

`chromosome_number`: counter for the number of pairs that were created.

`gen_number`: counter for the the number of generations.

In [4]:
algo_information = {
        'ProbabilityOfCrossover': 1,
        'ProbabilityOfMutation': 0.2,
        'Population': 30,
        'Generation': 10,
        'kfold': 3,
        'gene': starting_gene,
        'range_for_alpha': (0.0, 1.0),
        'range_for_binarization': (0.0, 1.0),
        'L': len(X),
        'X': X,
        'Y': y,
        'Winners': [],
        'chromosome_number': 1,
        'gen_number': 1
        }

Our first step is to initialize a population of random solutions. In our case this is a random amount binary values for $\alpha$ and binarize $\beta$.

In [5]:
def create_chromosomes_from(algo_info):
    """
    Takes the starting chromosome bit string, and creates a group of chromosome
    replicas, based on the population amount
    
    Parameters
    ----------
    
    `algo_info`: <dict> algorithm information
    
    Return
    ------
    
    [chromosomes]
    
    """
    print("Created chromosome # ", algo_info['chromosome_number'])
    print("IN GENERATION #", algo_info['gen_number'])
    pop = algo_info['Population']
    n_list = np.empty((0, len(algo_info['gene'])))
    for i in range(pop):
        rd.shuffle(algo_info['gene'])
        n_list = np.vstack((n_list, algo_info['gene']))
    algo_info['chromosome_number'] += 1
    if "NewPopulation" in algo_info.keys():
        rd.shuffle(algo_info['NewPopulation'])
        newpop = np.vstack((n_list, algo_info['NewPopulation']))
        return newpop 
    else:    
        return n_list

Our binaries need a way to be converted into actual numbers that humans can read so that we can use them in our model. So we created a way to decode binaries back into whole numbers with the appropriate precision.

In [6]:
def decode_chromosome(a, b, param_as_bit):
    """
    This function is design to decode a chromosome. It is derived from the formula
    above, and needs a upper range and a lower range given so that it can decode
    the binary bit string and return a decoded value.
    
    Parameters
    ---------
    
    `a` <float/int>lower value of the desired search space
    `b`<float/int> high end of the search space.
    `params_as_bit`<list> a list of ones and zeros representing the parameter as
    a binary chromosome.
    
    Return
    ------
    
    <float> Decoded Chromosome
    """
    l = len(param_as_bit)
    precision = (b - a) / ((2**l) - 1)
    z = 0
    t = 1
    num_sum = 0
    
    for i in range(l):
        num = param_as_bit[-t]*2**z
        num_sum += num
        t += 1
        z += 1
        
    decoded_value = (num_sum*precision) + a
    return decoded_value

In order for us to determine the best solution we need to have a selection process. Our selection process consists of the top 3 solutions or chromosomes that reduce the error of the model the most. These top 3 models are selected for a chance to go through crossover and mutation.

In [7]:
 def selection(algo_info):
    """
    This function runs the selection criteria from the group of different chromosomes
    in which the problem is centered. It grabs random indexes from the chromosomes and
    selects those genes and returns them ass candidates. In this case we are testing
    3 folds, so 3 candidates are returned.
    
    Parameter
    --------
    `algorithm information`: <dict> map containing all the starting info for the algo
    
    Return
    ------
    
    candidates: <list>
    """
    chromosomes = create_chromosomes_from(algo_info)
    indexes = np.unique(np.random.randint(0, len(chromosomes), size=3))
    while len(indexes) < 3:
        indexes = np.unique(np.random.randint(0, len(chromosomes), size=3))
        if len(indexes) == 3:
            break
    [w1, w2, w3] = indexes 
    warrior_1 = chromosomes[w1]
    warrior_2 = chromosomes[w2]
    warrior_3 = chromosomes[w3]
    candidates = [warrior_1, warrior_2, warrior_3]
    return candidates

In order to know if a candidate was good or not we need to calculate a fitness value. In our fitness value function does just that.

In [8]:
def fitness_value(algo_info):
    """
    Calculates the fitenss values for alpha and binarize for each participant in the 
    selection process. This function returns the candidate gene and its associated
    decoded value to the user for alpha and binarize
    
    Parameter
    --------
    `algorithm_informaiton`: <dict> information about algorithm.
    
    Return
    ------
    [{values}]: <list>
    """
    candidates = selection(algo_info)
    values = []
    
    for candidate in candidates:
        alpha_chromosome = candidate[0:-15]
        binarize_chromosome = candidate[15:len(candidate)]
        alpha_a = algo_info['range_for_alpha'][0]
        alpha_b = algo_info['range_for_alpha'][1]
        binarize_a = algo_info['range_for_binarization'][0]
        binarize_b = algo_info['range_for_binarization'][1]
        decoded_alpha = decode_chromosome(alpha_a, alpha_b , alpha_chromosome)
        decoded_binarize = decode_chromosome(binarize_a, binarize_b, binarize_chromosome)
        info = dict(Gene=candidate, Alpha=decoded_alpha, Binarize=decoded_binarize)
        values.append(info)
    return values

Our fitness function is to determine which values produce the lowest error for our dataset given the Bernoulli Naive Bayes model. We can't just simply take face value because we are doing this fitness test across multiple folds of the data. As a result we will take the average of all the folds and use that final value as the fitness measure.

In [9]:
def fitness(algo_info):
    """
    This function is the objective function which calculates the svm error for
    the specific gene. It returns each gene with its error function on the model.
    This giives a good idea of hor the decoded C, and Gamma value performed within
    the model.
    
    Parameter
    --------
    `algorithm_informaiton`: <dict> information about algorithm.
    
    Return
    ------
    [{values}]: <list>
    """
    values = fitness_value(algo_info)
    X = algo_info['X']
    Y = algo_info['Y']
    
    for value in values:
        kf = KFold(algo_info['kfold'])
        sum_of_scores = 0
        for train_index, test_index in kf.split(X):
            print("TRAIN:", train_index, "TEST: ", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            model = BernoulliNB(alpha=value['Alpha'], binarize=value['Binarize'])
            model.fit(X_train, y_train)
            model.predict(X_test)
            accuracy = model.score(X_test, y_test)
            gene_best_value = 1 - (accuracy)
            sum_of_scores += gene_best_value
        performance = sum_of_scores/algo_info['kfold']
        value['error'] = performance
    best_values = tournament_selection(values)
    return best_values

Only the best values get to compete to go into our crossover lottery. So our next function of the framework is designed to use the tournament style selection where the best values that lower the error rate are considered as parents.

In [10]:
def tournament_selection(fitness_values):
    parent_1_index = min(range(len(fitness_values)), key=lambda index: fitness_values[index]['error'])
    parent_1 = fitness_values[parent_1_index]
    del fitness_values[parent_1_index]
    parent_2_index = min(range(len(fitness_values)), key=lambda index: fitness_values[index]['error'])
    parent_2 = fitness_values[parent_2_index]
    del fitness_values[parent_2_index]
    parents = [parent_1, parent_2]
    return parents

The parents (or best solutions) go through a crossover process. This crossover process is simply a small exchange between the two best solutions. It is important that these changes are slight so as to not deviate too far from whatever that best value was for the parents. This crossover creates exchange of information only involving the best traits from the parents. A diagram is listed below to illustrate....

![Crossover Representation](images/crossover_image.png "Title")

In [11]:
def crossover(algo_info):
    parents = fitness(algo_info)
    algo_info['Winners'].append(parents[0])
    parent_1 = parents[0]['Gene']
    parent_2 = parents[1]['Gene']
    prob = np.random.rand()
    if prob < algo_info['ProbabilityOfCrossover']:
        cross_section_1 = np.random.randint(0, len(algo_info['gene']))
        cross_section_2 = np.random.randint(0, len(algo_info['gene']))
        
        while cross_section_1 == cross_section_2:
            cross_section_2 = np.random.randint(0, len(algo_info['gene']))
        if cross_section_1 < cross_section_2:
            middle_segment_1 = parent_1[cross_section_1:cross_section_2+1]
            middle_segment_2 = parent_2[cross_section_1:cross_section_2+1]
            
            first_segment_1 = parent_1[:cross_section_1]
            second_segment_1 = parent_1[cross_section_2+1:]
            
            first_segment_2 = parent_2[:cross_section_1]
            second_segment_2 = parent_2[cross_section_2+1:]
            
            child_1_gene = np.concatenate([first_segment_1, middle_segment_1, second_segment_1]) 
            child_2_gene = np.concatenate([first_segment_2, middle_segment_2, second_segment_2])
        else:
            middle_segment_1 = parent_1[cross_section_2:cross_section_1+1]
            middle_segment_2 = parent_2[cross_section_2:cross_section_1+1]
            
            first_segment_1 = parent_1[:cross_section_2]
            second_segment_1 = parent_1[cross_section_1+1:]
            
            first_segment_2 = parent_2[:cross_section_2]
            second_segment_2 = parent_2[cross_section_1+1:]
            
            child_1_gene = np.concatenate([first_segment_1, middle_segment_1, second_segment_1]) 
            child_2_gene = np.concatenate([first_segment_2, middle_segment_2, second_segment_2])
        return(child_1_gene, child_2_gene)
    else:
        child_1_gene = parent_1
        child_2_gene = parent_2
        return[child_1_gene, child_2_gene]
        

This crossover produces entirely new solutions, or genes which can be randomly tried for a mutation. The mutation is determined by random chance. In our case there is only a 20% chance there will be a mutation. If the chance of mutation is greator than 20% then each value in the binary gen will be changed. If it was 1 it will become a 0. If 0 it will become a one. This process will give an entirely mutated solution to try in the next generation.

![Mutation Representation](images/mutation.png "Title")

In [12]:
def mutate(algo_info):
   children =  crossover(algo_info)
   index = 0
   for child in children:
       for i in child:
           prob = np.random.rand()
           if prob < algo_info['ProbabilityOfMutation']:
               if child[index] == 0:
                   child[index] = 1
               else:
                   child[index] = 0
               index += 1
           else:
               pass
   return children

Our execution function takes all these ideas in consideration and can be coded as the following...

In [13]:
def execute(algo_info):
    print("EXECUTING GENTIC ALGO, In search for the best parameters.....")
    for i in tqdm(range(algo_info['Generation'])):
        algo_info['NewPopulation'] = np.empty((0,len(algo_info['gene'])))
        print("GENERATION # ", algo_info['gen_number'])
        for j in range(int(algo_info['Population']/2)):
            child1, child2 = mutate(algo_info)
            algo_info['NewPopulation'] = np.vstack((child1, child2, algo_info['NewPopulation']))        
        algo_info['gen_number'] += 1
        print("STARTING GENERATION #", algo_info['gen_number'])
    best_index = min(range(len(algo_info['Winners'])), key=lambda index: algo_info['Winners'][index]['error'])
    print("GENETIC ALGORITHMS BEST PARAMETERS AFTER GENERATION: ", algo_info['gen_number'])
    print(algo_info['Winners'][best_index])

Finally, our search takes a long time if the work is not parallelized. In an effort to speed up the search process we parallelized the work so that our search could make use of mutiple processes on the machine. The next grid executes our algorithm.

In [14]:
with Pool(processes=algo_information['Population']) as pool:
    pool.map(execute, [algo_information])

EXECUTING GENTIC ALGO, In search for the best parameters.....


  0%|          | 0/10 [00:00<?, ?it/s]

GENERATION #  1
Created chromosome #  1
IN GENERATION # 1
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177

TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
Created chromosome #  10
IN GENERATION # 1
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 8894

 10%|█         | 1/10 [10:37<1:35:35, 637.23s/it]

...

 90%|█████████ | 9/10 [1:36:32<10:43, 643.01s/it]

GENERATION #  10
Created chromosome #  136
IN GENERATION # 10
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900

TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
TRAIN: [ 88950  88951  88952 ... 266845 266846 266847] TEST:  [    0     1     2 ... 88947 88948 88949]
TRAIN: [     0      1      2 ... 266845 266846 266847] TEST:  [ 88950  88951  88952 ... 177896 177897 177898]
TRAIN: [     0      1      2 ... 177896 177897 177898] TEST:  [177899 177900 177901 ... 266845 266846 266847]
Created chromosome #  145
IN

100%|██████████| 10/10 [1:47:16<00:00, 643.12s/it]


GENETIC ALGORITHMS BEST PARAMETERS AFTER GENERATION:  11
{'Gene': array([1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.]), 'Alpha': 0.9036835840937528, 'Binarize': 0.4173100985747856, 'error': 0.14787827745820784}


#### The Results

We find after using our genetic algorithm to perform our parameter search we get the following...

|       BernoulliNB     | Best Params    |
|:---------------------:|:---------:|
| Best $\alpha$          | 0.9036835840937528|
| Best $\beta$          | 0.4173100985747856 |
|Estimated Error        | 0.14787827745820784 |
|Estimated Accuracy     | 0.8521217225417922 |
|Algorithm Runtime      | 1:47;16 |



The output tells us that if we choose to use the `BernoulliNB` model our best parameters to try are `0.92` for $\alpha$ and for binarize $\beta$ 0.15. Using these values on our dataset we can expect to get an estimated error of `0.15`. The accuracy that we can expect to get if we use these values is 85%. The runtime for this specific dataset to train on all of our 266,000 records takes 2 hours and 5 minutes until completion. With this information we can determine whether or not it is even worth moving forward with this model based on runtime and estimated error we might get.

This genetic algorithm framework can be applied to ANY model. In our example we used BernoulliNB, but the same could be tried with `SVC`, `DecisionTrees` and a host of others. In the future we expect to extend this framework so that it could become more user friendly, and actually recommend to us what is the best model to choose from!